# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pwd

/home/gongai/projects/wgong/py4kids/lesson-18-ai/crewai/gongai


In [2]:
from IPython.display import Markdown, display
import os
from time import time
import json
from pprint import pprint

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, MDXSearchTool,
    DirectoryReadTool, FileReadTool, BaseTool
)

from langchain_openai import ChatOpenAI

from pydantic import BaseModel

from api_key_store import ApiKeyStore


# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from helper import *

## Setup

### Inputs

In [4]:
# inputs json doc
# 
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """
        Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
        managing remote and in-office teams, and expert in multiple programming languages and frameworks. 
        He holds an MBA and a strong background in AI and data science. Noah has successfully led
        major tech initiatives and startups, proving his ability to drive innovation and growth in the tech industry. 
        Ideal for leadership roles that require a strategic and innovative approach.
    """
}

### Outputs

In [5]:
folder_name = "job-app"
create_folder(folder_name)

In [6]:
file_in_draft_resume = Path(folder_name) / 'fake_resume.md'
file_out_interview_material = Path(folder_name) / "interview_materials.md"
file_out_tailored_resume = Path(folder_name) / "tailored_resume.md"

In [7]:
file_in_draft_resume, str(file_in_draft_resume), file_in_draft_resume.name

(PosixPath('job-app/fake_resume.md'),
 'job-app/fake_resume.md',
 'fake_resume.md')

### LLM and keys

- https://docs.crewai.com/how-to/LLM-Connections/#ollama-integration

In [8]:
s = ApiKeyStore()

# os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI/Yiwen")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

# use Ollama
model_name = "crewai-llama3" # "gpt-3.5-turbo"
ollama_url = "http://localhost:11434/v1"
os.environ["OPENAI_API_BASE"] = ollama_url
os.environ["OPENAI_MODEL_NAME"] = model_name   # gpt-4 was default, throws rate-limit error 
os.environ["OPENAI_API_KEY"] = "NA"   # no key for Ollama

In [9]:
llm = ChatOpenAI(
    model = model_name,
    base_url = ollama_url
)

## Define Tools

In [10]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=file_in_draft_resume)
semantic_search_resume = MDXSearchTool(mdx=str(file_in_draft_resume))

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [11]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Create Agents

In [12]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on  job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory="""
        "As a Job Researcher, your prowess in  navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary  qualifications and skills sought "
        "by employers, forming the foundation for  effective application tailoring."
    """.replace('"',''),
    llm = llm,  # use ollama LLM
)

In [13]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants  to help them stand out in the job market",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "Equipped with analytical prowess, you dissect  and synthesize information "
        "from diverse sources to craft comprehensive  personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    """.replace('"',''),
    llm = llm,  # use ollama LLM
)

In [14]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a  resume stand out in the job market.",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "With a strategic mind and an eye for detail, you  excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they  resonate perfectly with the job's requirements."
    """.replace('"',''),
    llm = llm,  # use ollama LLM
)

In [15]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points  based on the resume and job requirements",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "Your role is crucial in anticipating the dynamics of  interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success,  ensuring they can confidently address all aspects of the "
        "job they are applying for."
    """.replace('"',''),
    llm = llm,  # use ollama LLM
)

## Create Tasks

In [16]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description="""
        "Analyze the job posting URL provided ({job_posting_url})  to extract required key skills, experiences, and qualifications. "
        "Use the tools to gather content and identify  and categorize the requirements."
    """.replace('"',''),
    expected_output="A structured list of job requirements, including necessary skills, qualifications, and experiences.",
    agent=researcher,
    async_execution=True
)

In [17]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description="""
        "Compile a detailed personal and professional profile  using the GitHub ({github_url}) URLs, "
        "and personal write-up ({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    """.replace('"',''),
    expected_output="""
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and communication style."
    """.replace('"',''),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [18]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
# depends on research_task, profile_task
resume_strategy_task = Task(
    description="""
        "Using the profile and job requirements obtained from previous tasks,"
        " tailor the resume to highlight the most relevant areas. "
        "Employ tools to adjust and enhance the resume content. "
        "Make sure this is the best resume even but don't make up any information. "
        "Update every section,  inlcuding the initial summary, work experience, skills, and education. "
        "All to better reflrect the candidates  abilities and how it matches the job posting."
    """.replace('"',''),
    expected_output="""
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    """.replace('"',''),
    output_file=str(file_out_tailored_resume),
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [19]:
# Task for Interview Preparer Agent: Develop Interview Materials
# depends on research_task, profile_task, resume_strategy_task
interview_preparation_task = Task(
    description="""
        "Create a set of potential interview questions and talking  points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion points. "
        "Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume  and how it matches the job posting."
    """.replace('"',''),
    expected_output="""
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    """.replace('"',''),
    output_file=str(file_out_interview_material),
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Create the Crew

In [20]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Run the Crew Workflow

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [21]:
### this execution will take a few minutes to run
ts_start = time()
result = job_application_crew.kickoff(inputs=job_application_inputs)
ts_stop = time()

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: 
        Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund)  to extract required key skills, experiences, and qualifications. 
        Use the tools to gather content and identify  and categorize the requirements.
    
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: 
        Compile a detailed personal and professional profile  using the GitHub (https://github.com/joaomdmoura) URLs, 
        and personal write-up (
        Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
        managing remote and in-office teams, and expert in multiple programming languages and frameworks. 
        He holds an MBA and a strong background in AI and data science. Noah has successfully led
    

In [25]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")   # Elapsed time: 53.83 sec

# on 2024-06-29
# 26.14 sec  using gpt-35-turbo
# 3092.06 sec  using crewai-llama3





Elapsed time: 3092.06 sec


- Dislplay the generated `tailored_resume.md` file.

In [23]:

display(Markdown(file_out_tailored_resume))

Based on job-app/fake_resume.md content and previous tasks, Personal Profiler for Engineers will update Noah Williams' resume to highlight the most relevant areas. The updated resume will showcase his experience as a Software Engineering Leader, emphasizing his leadership skills in managing remote teams, process innovation, and team collaboration. It will also feature his expertise in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir.

Here are some key points that should be included in the revised resume:

* Transforming the engineering division into a revenue pillar at DataKernel
* Spearheading the integration of AI technologies and scalable vector databases at DataKernel
* Defining long-term strategic initiatives for adopting AI technologies
* Leading diverse teams across multiple time zones
* Recruiting, mentoring, and retaining top talent
* Founding InnovPet and overseeing product development
* Building the initial version of the product using MongoDB

The resume should also highlight his entrepreneurial skills through his work at InnovPet. Additionally, it would be beneficial to emphasize Noah's technical expertise in programming languages and frameworks.

Overall, Personal Profiler for Engineers will update the resume to create a comprehensive summary that showcases Noah Williams' technical, strategic, and leadership abilities as a senior executive in software engineering.

- Dislplay the generated `interview_materials.md` file.

In [24]:
display(Markdown(file_out_interview_material))

** Below is the revised resume for Noah Williams:

**Summary:**

As a distinguished Software Engineering Leader with 18 years of experience in technology, I have consistently excelled in leading both remote and in-office engineering teams. My expertise spans software development, process innovation, team collaboration, and strategic initiatives. I am highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, with deep expertise in various front-end frameworks.

**Professional Experience:**

**DataKernel (remote), Director of Software Engineering**
2022 - Present

* Transformed the engineering division into a key revenue pillar, rapidly expanding the customer base
* Spearheaded AI technology integration and scalable vector databases, enhancing product capabilities and market positioning
* Defined long-term strategic initiatives for adopting AI technologies, setting new industry benchmarks

**DataKernel (remote), Senior Software Engineering Manager**
2019 - 2022

* Directed engineering strategy and operations, shaping the company's technological trajectory
* Managed diverse teams across multiple time zones in North America and Europe, fostering transparency and mutual respect
* Recruited, mentored, and retained top talent, creating a culture of continuous improvement

**InnovPet (remote), Founder & CEO**
2019 - 2022

* Founded a startup focused on IoT solutions for pet care, including GPS tracking collars that enhanced pet safety and owner peace of mind
* Oversaw product development from concept through execution, collaborating with engineering teams and marketing partners
* Set up an advisory board, established production facilities overseas, and navigated the company through initial funding

**Education:**

MBA in Information Technology (London Business School)
Advanced Leadership Techniques (University of London - Certification)
Data Science Specialization (Coursera - John Hopkins University - Certification)
B.Sc. in Computer Science (University of Edinburgh)

This revised resume highlights Noah Williams' technical, strategic, and leadership abilities as a senior executive in software engineering.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)